In [12]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZvM6LUyWI7hiVw6K64bt")
project = rf.workspace("capitaletech-wrnth").project("annotation-moxcs")
version = project.version(2)
dataset = version.download("yolov8")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to cvparsing-2 in yolov9:: 100%|██████████| 2344/2344 [00:00<00:00, 5118.00it/s]


In [1]:
!pip install ultralytics

In [6]:
yaml_text = """train: /cvparsing-2/train/images
val: /cvparsing-2/valid/images
test: /cvparsing-2/test/images

nc: 14
names: ['Achievement', 'Certifications', 'Community', 'Contact', 'Education', 'Experience', 'Interests', 'Languages', 'Name', 'Profil', 'Projects', 'image', 'resume', 'skills']"""

with open("./data.yaml", 'w') as file:
    file.write(yaml_text),

# To display the content of the file, you can use the 'cat' command like this:
# %cat /kaggle/working/data.yaml


In [20]:
!yolo train model=yolov9c.yaml data=D:/FU/DAT/src/notebook/datasets/data.yaml epochs=100 imgsz=640 device=0

In [1]:
# %pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.90  Python-3.11.9 torch-2.4.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Setup complete  (20 CPUs, 15.7 GB RAM, 33.9/97.7 GB disk)


In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests

# load image from the IAM database
# url = 'https://fki.tic.heia-fr.ch/static/img/a01-122-02-00.jpg'
image = Image.open(r'./images.png').convert("RGB")

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')
pixel_values = processor(images=image, return_tensors="pt").pixel_values

generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


d:\FU\DAT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\FU\DAT\.venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\htbqn\.cache\huggingface\hub\models--microsoft--trocr-base-handwritten. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this arti

In [3]:
generated_ids

tensor([[  2, 288, 321,   2]])

In [6]:
import onnxruntime as ort
import numpy as np
from PIL import Image

# Load the ONNX model
model_path = "../model/section_detection.onnx"
session = ort.InferenceSession(model_path)

# Load and preprocess the image
image_path = 'D:/FU/DAT/src/notebook/datasets/train/images/1629756071561_jpg.rf.05f192117b5f0f8125474abdf3392f72.jpg'
image = Image.open(image_path)
image_data = np.array(image).astype('float32').transpose(2, 0, 1)


In [7]:
image_data = np.expand_dims(image_data, axis=0)
image_data.shape

(1, 3, 640, 640)

In [8]:
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [9]:
result = session.run([output_name], {input_name: image_data})[0]

In [33]:
result[0].shape

(18, 8400)

In [10]:
from ultralytics import YOLO

# Load the YOLOv8 model'

# Load the exported ONNX model
onnx_model = YOLO("../model/section_detection.onnx")

# Run inference
results = onnx_model("D:/FU/DAT/src/notebook/datasets/train/images/1629756071561_jpg.rf.05f192117b5f0f8125474abdf3392f72.jpg")

WARNING  Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading ..\model\section_detection.onnx for ONNX Runtime inference...

image 1/1 D:\FU\DAT\src\notebook\datasets\train\images\1629756071561_jpg.rf.05f192117b5f0f8125474abdf3392f72.jpg: 640x640 2 Achievements, 147.6ms
Speed: 2.5ms preprocess, 147.6ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)


In [11]:
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

In [2]:
!roboflow login


You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).


In [4]:
from inference import get_model
import supervision as sv
import cv2

# define the image url to use for inference
image_file = "taylor-swift-album-1989.jpeg"
image = cv2.imread(image_file)

# load a pre-trained yolov8n model
model = get_model(model_id="annotation-moxcs/2")

# run inference on our chosen image, image can be a url, a numpy array, a PIL image, etc.
results = model.infer(image)[0]

# load the results into the supervision Detections api
detections = sv.Detections.from_inference(results)

# create supervision annotators
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

# annotate the image with our inference results
annotated_image = bounding_box_annotator.annotate(
    scene=image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

# display the image
sv.plot_image(annotated_image)

RoboflowAPINotAuthorizedError: Unauthorized access to roboflow API - check API key. Visit https://docs.roboflow.com/api-reference/authentication#retrieve-an-api-key to learn how to retrieve one.